In [2]:
# import required packages
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from IPython.display import display
from PIL import Image
from platform import python_version
import io
import math
import csv
import numpy as np
from csv import DictWriter
import re
import os
from pathlib import Path
import itertools
import shutil
from statistics import mode

import pytesseract
# Provide the path of the install location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\MatthewToberman\miniconda3\envs\agisoft_testing\Library\bin\tesseract.exe'
tessdata_dir_config = r'--tessdata-dir "C:\Users\MatthewToberman\miniconda3\pkgs\tesseract-5.0.1-h17c68af_0\share\tessdata"'




In [12]:
# Enter path of highest tier of video files 
# Enter path of highest tier of video files 
video_files_highest_directory = Path(r"W:\CHEVRON\AA_DONT_CHANGE_Chevron2022\GS-MIKE_FLARE")
video_file_paths_mpg = [str(pp) for pp in video_files_highest_directory.glob("**/*.mpg")]
video_file_paths_asf = [str(pp) for pp in video_files_highest_directory.glob("**/*.asf")]
video_file_paths = video_file_paths_mpg + video_file_paths_asf


display_plots = 0

for video_file_name in video_file_paths:

    image_file_path = video_file_name.split('\\')
    cropped_image_folder = ('\\'.join(map(str, image_file_path[0:-1])))
    cropped_image_folder = cropped_image_folder + '\\FRAMES_' + image_file_path[-1].split('@')[-1].split('.')[0]
    os.mkdir(cropped_image_folder)
    # Read the video from specified path this could clearly be changed to a file open box
    cap = cv2.VideoCapture(video_file_name)  
    # find length of entire video in frames
    total_frame_number = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # find frame rate of video
    frame_rate=cap.get(cv2.CAP_PROP_FPS)

    #  find the time at the end of video so can use this if entire video is required
    video_end_time_mins,video_end_time_secs  = divmod(math.floor(total_frame_number / frame_rate), 60)

    # Enter video file start time and end time and frame interval for frame grab 
    start_time_mins=0
    start_time_secs=1
    end_time_mins=video_end_time_mins
    end_time_secs=video_end_time_secs
    frame_interval=50

    # find start and end frames coorresponding to start and end times
    start_frame=math.floor((start_time_mins*60*frame_rate)+(start_time_secs*frame_rate))
    end_frame=math.floor((end_time_mins*60*frame_rate)+(end_time_secs*frame_rate))


    frame_counter = 0
    text_depth_nums = []
    text_meta_datas = []
    # Loop through specified frames 
    # for i in range(1,2,1):
    for i in range(start_frame,end_frame,frame_interval):
        
    
        # choose specified frame
        cap.set(cv2.CAP_PROP_POS_FRAMES,i)
        # read specified frame
        ret, frame = cap.read()
        if isinstance(frame, np.ndarray):
            # First crop banner text from image as otherwise this can be included in the blurr detection process and make the image appear in focus. These banners will need to be removed for agisoft proccesing anyhow
            #set pixel value of top of cropped image
            vert_crop_start = 1
            #set pixel value of bottom of cropped image
            vert_crop_end = frame.shape[0]
            horz_crop_start = 1
            horz_crop_end = frame.shape[1]
            # Crop frame
            crop_frame = frame[vert_crop_start:vert_crop_end, horz_crop_start:horz_crop_end]
            # create time stamp for frame file name
            fractional_mins, whole_mins = math.modf((i/frame_rate)/60)

            # # select area of image with required text for meta data only from first frame shod change this until only an acceptable value is recieved 
    
            text_frame_meta_data_horz_start = 0 
            text_frame_meta_data_horz_end = 800
            text_frame_meta_data_vert_start = 500
            text_frame_meta_data_vert_end = 710
            text_frame_meta_data =  frame[ text_frame_meta_data_vert_start:text_frame_meta_data_vert_end,text_frame_meta_data_horz_start:text_frame_meta_data_horz_end]
            text_meta_data = pytesseract.image_to_string(text_frame_meta_data, config=tessdata_dir_config)
            if len(text_meta_data) < 4:
                text_meta_data = 'notextfound'
            else:
                text_meta_data =  ''.join(e for e in text_meta_data if e.isalnum())
                text_meta_data =  ''.join(e for e in text_meta_data if e.isalnum())
            text_meta_datas.append(text_meta_data)
            
            # select area of image with required text for depth data
            text_frame_depth_data_horz_start = 440
            text_frame_depth_data_horz_end = 700
            text_frame_depth_data_vert_start = 1
            text_frame_depth_data_vert_end = 60
            text_frame_depth_data =  frame[ text_frame_depth_data_vert_start:text_frame_depth_data_vert_end,text_frame_depth_data_horz_start:text_frame_depth_data_horz_end]
            text_depth_data = pytesseract.image_to_string(text_frame_depth_data, config=tessdata_dir_config)        
            if len(text_depth_data) > 0:
                if len((re.findall("\d+", text_depth_data))) >0 :
                    text_depth_num = (re.findall("\d+", text_depth_data))
                    text_depth_num = (''.join(text_depth_num))
                    if len(text_depth_num) > 0:
                        text_depth_num =int(text_depth_num)
                        if text_depth_num < 5 or text_depth_num > 100:
                            text_depth_num = 'na'

                else:
                    text_depth_num = 'na' 
            else:
                text_depth_num = 'na'    
            text_depth_nums.append(text_depth_num)



            if display_plots ==1:
                fig,ax =plt.subplots() 
                ax.imshow(frame)
                plt.title('Frame')
                plt.show()

                # show selected area for testing
                plt.imshow(text_frame_meta_data)
                plt.title('Meta Data')
                plt.show()
                print(text_meta_data)
                # show selected area for testing
                plt.imshow(text_frame_depth_data)
                plt.title('Depth Data')
                plt.show()
            if display_plots ==1:
                print(text_depth_num)


            # create and save jpegs
            frame_image_name=cropped_image_folder + '/' 'Frame_'+str(i) + '_' + str(round(whole_mins)) + 'mins' + '_' + str(round(fractional_mins*60)) + 'secs_'  + '.jpg'
            cv2.imwrite(frame_image_name,crop_frame)
        #     frame_counter +=1
        
        
    text_depth_nums_no_nan = ([num for num in text_depth_nums if num != 'na' ])
    
    text_meta_datas_no_nan  = ([meta for meta in text_meta_datas if meta != 'notextfound'])
    if len(text_meta_datas_no_nan) < 2:
        text_meta_datas_no_nan = 'no_metaa'
        text_meta_datas_no_nan = 'no_metaa'

    if len(text_meta_datas_no_nan) >=  2:
        mode_text_meta_datas_no_nan = mode(text_meta_datas_no_nan)
    else :
        mode_text_meta_datas_no_nan = 'no_metaa'
    if mode_text_meta_datas_no_nan == '':
        mode_text_meta_datas_no_nan = 'no_metaa'
    if len(text_depth_nums_no_nan) > 0:
        shutil.move(cropped_image_folder ,'/'.join(cropped_image_folder.split('\\')[:-1]) + '/' + mode_text_meta_datas_no_nan[:-1]   + '_depth_'  + str(min(text_depth_nums_no_nan)) +'_' + str(max(text_depth_nums_no_nan)))
    else:
        shutil.move(cropped_image_folder ,'/'.join(cropped_image_folder.split('\\')[:-1]) + '/' + mode_text_meta_datas_no_nan[:-1]   + '_depth_NA')
    print(mode_text_meta_datas_no_nan)

GSMIKECaissonsSeawaterLift3
GSMIKECaissonsSeawaterLift3
GSMIKECaissonsSeawaterLift2
GSMIKECaissonsSeawaterLift1
GSMIKECaissonsSumpPile1
GSMIKECaissonsSumpPile2
GSMIKEElev3
GSMIKEElev3Anode27
GSMIKERowA3101
GSMIKEElev1E105
GSMIKEElev1E108
GSMIKEElev1E115
GSMIKEElev1E116
GSMIKEElev1E110
GSMIKEElev1E104
GSMIKEElev1Anode03
GSMIKEElev1Anode09
GSMIKEElev1E112
GSMIKEElev1E102
GSMIKEElev1E1134
GSMIKEElev1Anode11
GSMIKEElev1E101
GSMIKEElev1E111
GSMIKEElev1E114
GSMIKEElev1E113
GSMIKEElev1E101
GSMIKEElev2E202
GSMIKEElev2E212
GSMIKEElev2E201
GSMIKEElev2Anode20
GSMIKEElev3E302
GSMIKE
GSMIKERisersRowB16GLIGSMtoGSDAUXRC3
GSMIKERisersRowB16PRDGSMtoGSNAUX
GSMIKERisersRowB12PRDTKHtoGSMRC3
GSMIKERisersRowB4GLIGSMtoTKHRC3
GSMIKERisersRowB24EXPGSMtoTakPlemEXP
GSMIKERisersRowB6PRDTKItoGSMRC3
GSMIKERisersRowB4GLIGSMtoTKlRC3
GSMIKERisersRowB12WINGSMtoTKORC3
GSMIKERisersRowB8PRDWTKItoGSMRC3
GSMIKERisersRowB8PRDETKItoGSMRC3
GSMIKERisersRowB8PRDWTKHtoGSMRC3
GSMIKERisersRowB8PRDETKHtoGSMRC3
GSMIKERisersRowB6PRDTK